In [2]:
library(tidyverse)
library(WGCNA)

# Custom package
library(rutils)

-- Attaching packages --------------------------------------------------------------------------------------------------------- tidyverse 1.3.0 --

v ggplot2 3.3.3     v purrr   0.3.4
v tibble  3.0.6     v dplyr   1.0.4
v tidyr   1.1.2     v stringr 1.4.0
v readr   1.4.0     v forcats 0.5.1

-- Conflicts ------------------------------------------------------------------------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()

Loading required package: dynamicTreeCut

Loading required package: fastcluster


Attaching package: 'fastcluster'


The following object is masked from 'package:stats':

    hclust





Attaching package: 'WGCNA'


The following object is masked from 'package:stats':

    cor




In [3]:
enableWGCNAThreads()

Allowing parallel execution with up to 15 working processes.


In [4]:
# Define constants
dirs <- rutils::get_dev_directories(dev_paths_file = "../dev_paths.txt")
projects <- c("TCGA-CESC", "TCGA-UCS", "TCGA-UCEC", "TCGA-OV")
unified_dsets <- c("unified_cervical_data", "unified_uterine_data", "unified_uterine_endometrial_data")
matrisome_path <- paste0(dirs$data_dir, "/matrisome/matrisome_hs_masterlist.tsv")

event_code <- list("Alive" = 0, "Dead" = 1)
# covariate_cols <- c("figo_stage", "age_at_diagnosis", "race", "ethnicity")
dep_cols <- c("vital_status", "survival_time")

In [5]:
dset_idx <- 1
save_res <- TRUE

In [15]:
# Load and filter survival data
survival_path <- paste0(dirs$data_dir, "/", unified_dsets[dset_idx], "/survival_data.tsv")
survival_df <- load_survival_df(survival_path, event_code)
cutoff_df <- read_tsv(paste0(dirs$analysis_dir, "/survival/", unified_dsets[dset_idx], "_expression_cutoffs.tsv"))

filtered_survival_df <- survival_df %>%
    dplyr::select(one_of(c("sample_name", dep_cols))) %>%
    dplyr::filter(rowSums(is.na(.)) == 0)

# Load normalized matrisome count data
norm_matrisome_counts_df <- read_tsv(paste0(dirs$data_dir, "/", unified_dsets[dset_idx], "/norm_matrisome_counts.tsv"))
norm_matrisome_counts_t_df <- norm_matrisome_counts_df %>%
    dplyr::select(c("geneID", filtered_survival_df$sample_name)) %>%
    transpose_df("geneID", "sample_name")
# Combine survival data and normalized count data
joined_df <- filtered_survival_df %>%
    inner_join(norm_matrisome_counts_t_df, by = "sample_name") %>%
    # cannot have survival times of 0
    dplyr::filter(survival_time > 0)


-- Column specification --------------------------------------------------------------------------------------------------------------------------
cols(
  sample_name = col_character(),
  vital_status = col_character(),
  survival_time = col_double(),
  figo_stage = col_character(),
  days_to_last_follow_up = col_double(),
  days_to_death = col_double(),
  age_at_diagnosis = col_double(),
  age_at_index = col_double(),
  height = col_double(),
  weight = col_double(),
  bmi = col_double(),
  race = col_character(),
  ethnicity = col_character()
)



-- Column specification --------------------------------------------------------------------------------------------------------------------------
cols(
  geneID = col_character(),
  cutoff = col_double()
)



-- Column specification --------------------------------------------------------------------------------------------------------------------------
cols(
  .default = col_double(),
  geneID = col_character()
)
i Use `spec()` for the f

In [17]:
x <- joined_df[-c(1:3)] %>% as.matrix()
rownames(x) <- joined_df$sample_name

In [18]:
cts_df <- standardScreeningCensoredTime(
    datExpr = x,
    time = joined_df$survival_time,
    event = joined_df$vital_status,
    fastCalculation = FALSE,
    qValues = TRUE
) %>% 
    as_tibble() %>%
    dplyr::select(ID, pvalueLogrank, qvalueLogrank, HazardRatio) %>%
    dplyr::rename(geneID = ID, vital_pval = pvalueLogrank, vital_qval = qvalueLogrank, hr = HazardRatio)

Warning message in fitter(X, Y, istrat, offset, init, control, weights = weights, :
"Loglik converged before variable  1 ; coefficient may be infinite. "
Warning message in fitter(X, Y, istrat, offset, init, control, weights = weights, :
"Loglik converged before variable  1 ; coefficient may be infinite. "
Warning message in fitter(X, Y, istrat, offset, init, control, weights = weights, :
"Loglik converged before variable  1 ; coefficient may be infinite. "
Warning message in fitter(X, Y, istrat, offset, init, control, weights = weights, :
"Loglik converged before variable  1 ; coefficient may be infinite. "
Warning message in fitter(X, Y, istrat, offset, init, control, weights = weights, :
"Loglik converged before variable  1 ; coefficient may be infinite. "


In [8]:
if (save_res) {
    write_tsv(cts_df, paste0(dirs$analysis_dir, "/feature_selection/", unified_dsets[dset_idx], "_survival_cts_results.tsv"))
}